<a href="https://colab.research.google.com/github/moysesneva/HOJE29072024/blob/main/Calculo_Horas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install openpyxl  # Para arquivos Excel no formato .xlsx
!pip install xlrd      # Se estiver trabalhando com arquivos Excel no formato .xls
!pip install pandas openpyxl


In [2]:
import pandas as pd
import numpy as np
import calendar
import openpyxl

# Função para carregar os dados diretamente do arquivo
def carregar_dados():
    aulas_df = pd.read_excel('aulas.xlsx')
    feriados_df = pd.read_excel('feriados.xlsx')
    return aulas_df, feriados_df

# Função para contar os dias da semana em um mês específico
def contar_dias_mes(mes, ano):
    calendario_mes = calendar.monthcalendar(ano, mes)

    contagem_dias = {
        'Segunda': 0,
        'Terça': 0,
        'Quarta': 0,
        'Quinta': 0,
        'Sexta': 0,
        'Sábado': 0,
        'Domingo': 0
    }

    for semana in calendario_mes:
        for i, dia in enumerate(semana):
            if dia != 0:
                if i == 0:
                    contagem_dias['Segunda'] += 1
                elif i == 1:
                    contagem_dias['Terça'] += 1
                elif i == 2:
                    contagem_dias['Quarta'] += 1
                elif i == 3:
                    contagem_dias['Quinta'] += 1
                elif i == 4:
                    contagem_dias['Sexta'] += 1
                elif i == 5:
                    contagem_dias['Sábado'] += 1
                elif i == 6:
                    contagem_dias['Domingo'] += 1

    return contagem_dias

# Função para padronizar os nomes dos dias da semana
def mapear_dia_semana(dia_completo):
    mapeamento = {
        'Segunda-feira': 'Segunda',
        'Terça-feira': 'Terça',
        'Quarta-feira': 'Quarta',
        'Quinta-feira': 'Quinta',
        'Sexta-feira': 'Sexta',
        'Sábado': 'Sábado',
        'Domingo': 'Domingo'
    }
    return mapeamento.get(dia_completo, dia_completo)

# Função para formatar valores monetários em reais
def formatar_moeda(valor):
    return f"R$ {valor:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

# Função para calcular a remuneração
def calcular_remuneracao(aulas_df, feriados_df, contagem_dias, valor_hora_extra):
    relatorio = []
    professores = aulas_df['Nome'].unique()  # Obter a lista de professores

    for professor in professores:
        print(f"\nColetando dados para o professor {professor}:")

        # Filtrar as aulas do professor atual
        aulas_professor = aulas_df[aulas_df['Nome'] == professor]

        faltas = int(input(f"Digite o número de faltas para o professor {professor}: "))
        horas_extras = float(input(f"Digite o número de horas extraordinárias para o professor {professor}: ").replace(",", "."))

        for index, row in aulas_professor.iterrows():
            nome = row['Nome']
            turma = row['Turma']
            dia_da_semana = mapear_dia_semana(row['Dia_da_Semana'])  # Usar a função de mapeamento
            carga_horaria = row['Carga_Horaria_Diaria']
            valor_hora = row['Valor_Hora_Aula']

            # Obter o número de dias em que as aulas ocorrem no mês
            total_dias = contagem_dias.get(dia_da_semana, 0)

            # Calcular o total de horas ministradas (sem ajustes ainda)
            horas_ministradas = total_dias * carga_horaria

            # Ajustar horas ministradas para faltas e adicionar horas extras
            horas_totais = horas_ministradas + horas_extras - faltas

            # Garantir que o total de horas não seja negativo
            if horas_totais < 0:
                horas_totais = 0

            # Calcular o total a receber com base na fórmula corrigida
            total_receber = horas_totais * valor_hora

            # Armazenar os dados no relatório
            relatorio.append({
                'Nome': nome,
                'Turma': turma,
                'Total de Horas Ministradas': horas_ministradas,
                'Dias da Semana': dia_da_semana,
                'Carga Horária Diária': carga_horaria,
                'Valor da Hora/Aula': formatar_moeda(valor_hora),
                'Faltas': faltas,
                'Horas Extraordinárias': horas_extras,
                'Total a Receber': formatar_moeda(total_receber)
            })

    return pd.DataFrame(relatorio)

# Função para gerar o relatório em Excel
def gerar_relatorio_excel(relatorio_df, nome_arquivo='relatorio_remuneracao.xlsx'):
    relatorio_df.to_excel(nome_arquivo, index=False)
    print(f"Relatório gerado com sucesso: {nome_arquivo}")

# Função principal
def main():
    # Carregar dados
    aulas_df, feriados_df = carregar_dados()

    # Perguntar sobre o mês e ano
    mes = int(input("Digite o número do mês (1-12) para gerar o relatório: "))
    ano = int(input("Digite o ano: "))

    # Contar os dias úteis no mês especificado
    contagem_dias = contar_dias_mes(mes, ano)
    print("Contagem de dias no mês:", contagem_dias)

    # Coletar o valor da hora extraordinária uma vez
    valor_hora_extra = float(input("Digite o valor da hora extraordinária: ").replace(",", "."))

    # Calcular a remuneração
    relatorio_df = calcular_remuneracao(aulas_df, feriados_df, contagem_dias, valor_hora_extra)

    # Gerar o relatório em Excel
    gerar_relatorio_excel(relatorio_df)

# Executar a função principal
main()


Digite o número do mês (1-12) para gerar o relatório: 08
Digite o ano: 2024
Contagem de dias no mês: {'Segunda': 4, 'Terça': 4, 'Quarta': 4, 'Quinta': 5, 'Sexta': 5, 'Sábado': 5, 'Domingo': 4}
Digite o valor da hora extraordinária: 29,99

Coletando dados para o professor Beatriz Sampaio:
Digite o número de faltas para o professor Beatriz Sampaio: 0
Digite o número de horas extraordinárias para o professor Beatriz Sampaio: 0

Coletando dados para o professor Eduardo Kawahata Barreto:
Digite o número de faltas para o professor Eduardo Kawahata Barreto: 0
Digite o número de horas extraordinárias para o professor Eduardo Kawahata Barreto: 0

Coletando dados para o professor Jose Eduardo:
Digite o número de faltas para o professor Jose Eduardo: 0
Digite o número de horas extraordinárias para o professor Jose Eduardo: 0

Coletando dados para o professor Júlia Soares Takimoto:
Digite o número de faltas para o professor Júlia Soares Takimoto: 0
Digite o número de horas extraordinárias para o p